---
🔠 Natural Language Processing (CS60075) Autumn 2024, IIT Kharagpur

📃 Assignment 3 Part 1: [Hate Speech Classification using Few-shot Prompting](https://sites.google.com/view/nlp-cs-iit-kgp/assignments)

👦🏻 Author: [Prasanna Paithankar (21CS30065)](https://cse.iitkgp.ac.in/~prasannabp/)

---

##### 📚 Import Libraries and Resources

In [1]:
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score
from transformers import T5ForConditionalGeneration, T5Tokenizer

d:\Codes\Natural-Language-Processing-Aut-24\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##### 🤗 Load Flan-T5 Models

In [2]:
tokenizer_small = T5Tokenizer.from_pretrained("google/flan-t5-small")
model_small = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small")

input_text = "translate English to German: How old are you?"
input_ids = tokenizer_small(input_text, return_tensors="pt").input_ids

outputs = model_small.generate(input_ids)
print(tokenizer_small.decode(outputs[0]))

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


<pad> Wie ich er bitten?</s>


d:\Codes\Natural-Language-Processing-Aut-24\venv\Lib\site-packages\transformers\generation\utils.py:1375: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


In [3]:
tokenizer_base = T5Tokenizer.from_pretrained("google/flan-t5-base")
model_base = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

input_text = "translate English to German: How old are you?"
input_ids = tokenizer_base(input_text, return_tensors="pt").input_ids

outputs = model_base.generate(input_ids)
print(tokenizer_base.decode(outputs[0]))

<pad> Wie old sind Sie?</s>


##### 🧪 Load Dataset

In [4]:
data = pd.read_csv('dataset/NLP_ass_test.tsv',
                        sep='\t', header=None,
                        names=['text', 'label'], dtype={'text': str, 'label': str})

##### 💳 Define Prompt Template

In [5]:
def template(text):
    return (
        "Task: Classify statement into one of the 3 categories according to following directives:\n\n"
        "**1. Normal:**\n"
        "- A normal statement, doesn't incite hate or offense towards a group of people.\n"
        "- It does not demean, insult, or target any individual or group.\n\n"
        "**2. Hatespeech:**\n"
        "- A targeted statement aimed at provoking hatred or intending to cause harm towards a particular group of people based on factors such as religion, sexual orientation, caste.\n"
        "- It uses derogatory slurs or calls for harmful actions against a group.\n\n"
        "**3. Offensive:**\n"
        "- A targeted statement which might be judged as demeaning by a group of people. The statement might not be hateful or aim to incite violence, though.\n"
        "- It may include swearing or disrespectful comments without targeting protected characteristics.\n\n"
        "**Directives:**\n"
        "- Read the statement carefully.\n"
        "- Identify any use of slurs, derogatory terms, or profanity.\n"
        "- Determine if the statement targets a specific individual or group.\n"
        "- Decide whether it incites violence or expresses severe hatred (Hatespeech), is demeaning or contains profanity without severe hatred (Offensive), or is neutral (Normal).\n\n"
        f"**Classify :** \"{text}\"\n\n"
        "Follow the above points and classify the statement into one of the 3 categories, i.e., Normal, Hatespeech, Offensive.\n"
        "Output only a single word which is the category of the statement."
    )

##### 0️⃣-Shot Prompting

In [6]:
flan_t5_small_preds = []

for text in data['text']:
    prompt = template(text)
    input_ids = tokenizer_small(prompt, return_tensors='pt').input_ids
    outputs = model_small.generate(input_ids, max_length=10)

    prediction = tokenizer_small.decode(outputs[0], skip_special_tokens=True)
    flan_t5_small_preds.append(prediction.strip().lower())

In [7]:
with open('flan_t5_small_preds.txt', 'w') as f:
    for prediction in flan_t5_small_preds:
        f.write(f"{prediction}\n")

##### 🤏🏻 Small Model Evaluation

In [8]:
flan_t5_small_preds = []
with open('flan_t5_small_preds.txt', 'r') as f:
    for line in f:
        flan_t5_small_preds.append(line.strip())

labels = data['label'].str.lower().tolist()

print()
print("T5 Small Model Results:")
print("Test Accuracy:", accuracy_score(labels, flan_t5_small_preds))
print("F1 Score:", f1_score(labels, flan_t5_small_preds, average='macro'))


T5 Small Model Results:
Test Accuracy: 0.4311226611226611
F1 Score: 0.21645794511733395


In [9]:
for i in [3, 7, 11, 15, 19]:
    print("Statement:", data['text'][i])
    print("True Label:", labels[i])
    print("Predicted Label:", flan_t5_small_preds[i])
    print()

Statement: as much as i appreciate a format of open discussion i really just want a facebook that i can say beaner on
True Label: normal
Predicted Label: normal

Statement: they are going to figure out a way to kike bitcoin
True Label: hatespeech
Predicted Label: normal

Statement: i raped a sheep because it sneezed at a pig
True Label: normal
Predicted Label: normal

Statement: cheating is not the wave grow up you dumbass nigga
True Label: normal
Predicted Label: normal

Statement: i d rather get fisted by a nigger tbh
True Label: offensive
Predicted Label: normal



In [10]:
flan_t5_base_preds = []

for text in data['text']:
    prompt = template(text)
    input_ids = tokenizer_base(prompt, return_tensors='pt').input_ids
    outputs = model_base.generate(input_ids, max_length=10)
    prediction = tokenizer_base.decode(outputs[0], skip_special_tokens=True)
    flan_t5_base_preds.append(prediction.strip().lower())

In [11]:
with open('flan_t5_base_preds.txt', 'w') as f:
    for prediction in flan_t5_base_preds:
        f.write(f"{prediction}\n")

##### ⚾ Base Model Evaluation

In [12]:
flan_t5_base_preds = []
with open('flan_t5_base_preds.txt', 'r') as f:
    for line in f:
        flan_t5_base_preds.append(line.strip())

labels = data['label'].str.lower().tolist()

print()
print("T5 Base Model Results:")
print("Test Accuracy:", accuracy_score(labels, flan_t5_base_preds))
print("F1 Score:", f1_score(labels, flan_t5_base_preds, average='macro'))


T5 Base Model Results:
Test Accuracy: 0.453991683991684
F1 Score: 0.3443492814988748


In [13]:
for i in [3, 7, 11, 15, 19]:
    print("Statement:", data['text'][i])
    print("True Label:", labels[i])
    print("Predicted Label:", flan_t5_base_preds[i])
    print()

Statement: as much as i appreciate a format of open discussion i really just want a facebook that i can say beaner on
True Label: normal
Predicted Label: normal

Statement: they are going to figure out a way to kike bitcoin
True Label: hatespeech
Predicted Label: offensive

Statement: i raped a sheep because it sneezed at a pig
True Label: normal
Predicted Label: offensive

Statement: cheating is not the wave grow up you dumbass nigga
True Label: normal
Predicted Label: offensive

Statement: i d rather get fisted by a nigger tbh
True Label: offensive
Predicted Label: offensive



##### 👨‍👩‍👧‍👦 Few-Shot Prompting 

In [14]:
def selector(train_data):
    few_shot_examples = []
    for label in train_data['label'].unique():
        example = train_data[train_data['label'] == label].sample(1)

        few_shot_examples.append({
            'text': example['text'].values[0],
            'label': example['label'].values[0]
        })
    return few_shot_examples

##### 💳 Define Prompt Template

In [15]:
def few_shot_template(text, examples):
    template = (
        "Task: Classify statement into one of the 3 categories according to following directives:\n\n"
        "**1. Normal:**\n"
        "- A normal statement, doesn't incite hate or offense towards a group of people.\n"
        "- It does not demean, insult, or target any individual or group.\n\n"
        "**2. Hatespeech:**\n"
        "- A targeted statement aimed at provoking hatred or intending to cause harm towards a particular group of people based on factors such as religion, sexual orientation, caste.\n"
        "- It uses derogatory slurs or calls for harmful actions against a group.\n\n"
        "**3. Offensive:**\n"
        "- A targeted statement which might be judged as demeaning by a group of people. The statement might not be hateful or aim to incite violence, though.\n"
        "- It may include swearing or disrespectful comments without targeting protected characteristics.\n\n"
        "**Here are some examples:**\n"
    )

    for example in examples:
        template += f"\nStatement: \"{example['text']}\"\nCategory: {example['label'].capitalize()}\n"
        
    template += (
        f"**Classify :** \"{text}\"\n\n"
        "Follow the above points and classify the statement into one of the 3 categories, i.e., Normal, Hatespeech, Offensive.\n"
        "Output only a single word which is the category of the statement."
    )

    return template

In [16]:
train_data = pd.read_csv('dataset/NLP_ass_train.tsv',
                        sep='\t', header=None,
                        names=['text', 'label'], dtype={'text': str, 'label': str})

In [17]:
flan_t5_small_preds_few_shot = []

for text in data['text']:
    examples = selector(train_data)

    prompt = few_shot_template(text, examples)

    input_ids = tokenizer_small(prompt, return_tensors='pt').input_ids
    outputs = model_small.generate(
        input_ids,
        max_length=10,
        num_beams=5,
        early_stopping=True
    )
    prediction = tokenizer_small.decode(outputs[0], skip_special_tokens=True)
    flan_t5_small_preds_few_shot.append(prediction.strip().lower())

##### 🤏🏻 Small Model Evaluation

In [18]:
labels = data['label'].str.lower().tolist()

print()
print("T5 Small Model Results (Few-Shot):")
print("Test Accuracy:", accuracy_score(labels, flan_t5_small_preds_few_shot))
print("F1 Score:", f1_score(labels, flan_t5_small_preds_few_shot, average='macro'))


T5 Small Model Results (Few-Shot):
Test Accuracy: 0.443993724681684
F1 Score: 0.2643492813856739


In [19]:
flan_t5_base_preds_few_shot = []

for text in data['text']:
    examples = selector(train_data)

    prompt = few_shot_template(text, examples)

    input_ids = tokenizer_base(prompt, return_tensors='pt').input_ids
    outputs = model_small.generate(
        input_ids,
        max_length=10,
        num_beams=5,
        early_stopping=True
    )
    prediction = tokenizer_base.decode(outputs[0], skip_special_tokens=True)
    flan_t5_base_preds_few_shot.append(prediction.strip().lower())

##### ⚾ Base Model Evaluation

In [20]:
labels = data['label'].str.lower().tolist()

print()
print("T5 Base Model Results (Few-Shot):")
print("Test Accuracy:", accuracy_score(labels, flan_t5_base_preds_few_shot))
print("F1 Score:", f1_score(labels, flan_t5_base_preds_few_shot, average='macro'))


T5 Base Model Results (Few-Shot):
Test Accuracy: 0.478462724696882
F1 Score: 0.3843782853854732


***
Prasanna Paithankar (21CS30065)